# Module 4 - Analysing SNPs in the *Vgsc* gene

## Learning objectives

* Know what the *Vgsc* gene is, where it is located within the genome, and what transcripts are known
* Know what a SNP is and what effects SNPs can cause
* Know how to define sample cohorts for analysis of SNP allele frequencies
* Be able to discover amino acid substitutions within *Vgsc* and compute their frequencies in different cohorts 


## @@TODO

## Feedback

@@TODO how to submit feedback